In [1]:
import numpy as np
import pandas as pd
import wave
import librosa
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, RMSprop
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error

In [ ]:
! pip install noisereduce

In [16]:
import noisereduce as nr

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/data_ml5/

Mounted at /content/drive
/content/drive/MyDrive/data_ml5


In [17]:
train_split_df = pd.read_csv('train_split_Depression_AVEC2017.csv')
test_split_df = pd.read_csv('dev_split_Depression_AVEC2017.csv')

In [20]:
train_split_num = train_split_df[['Participant_ID']]['Participant_ID'].tolist()
test_split_num = test_split_df[['Participant_ID']]['Participant_ID'].tolist()
# train_split_clabel = train_split_df[['PHQ8_Binary']]['PHQ8_Binary'].tolist()
# test_split_clabel = test_split_df[['PHQ8_Binary']]['PHQ8_Binary'].tolist()
train_split_clabel = train_split_df[['PHQ8_Score']]['PHQ8_Score'].tolist()
test_split_clabel = test_split_df[['PHQ8_Score']]['PHQ8_Score'].tolist()

In [21]:
def extract_features(number, audio_features, target, audio_targets, mode):
    transcript = pd.read_csv('{0}_TRANSCRIPT.csv'.format(number), sep='\t').fillna('')
    global wave_data
    global sr
    wavefile = wave.open('{0}_AUDIO.wav'.format(number, 'r'))
    sr = wavefile.getframerate()
    nframes = wavefile.getnframes()

    wave_data = np.frombuffer(wavefile.readframes(nframes), dtype=np.short)
    reduced_noise = nr.reduce_noise(y=wave_data, sr=sr)
    '''
    plt.figure(figsize=(10, 4))
    # time = np.linspace(0, nframes/sr, num=nframes)
    time = np.arange(0, nframes) / sr
    plt.plot(time, wave_data)
    plt.title('Waveform of Audio File')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()
    plt.figure(figsize=(10, 4))
    # time = np.linspace(0, nframes/sr, num=nframes)
    time = np.arange(0, nframes) / sr
    plt.plot(time, reduced_noise)
    plt.title('Waveform of Audio File')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()
    '''
    time_range = []
    response = ''
    response_flag = False
    time_collect_flag = False
    start_time = 0
    stop_time = 0

    signal = []

    global counter_train

    for t in transcript.itertuples():
        if getattr(t,'speaker') == 'Ellie':
            continue
        elif getattr(t,'speaker') == 'Participant':
            if 'scrubbed_entry' in getattr(t,'value'):
                continue
            start_time = int(getattr(t,'start_time')*sr)
            stop_time = int(getattr(t,'stop_time')*sr)
            signal = np.hstack((signal, reduced_noise[start_time:stop_time].astype(np.float64)))

    clip = sr*15
    # if target == 1 and mode == 'train':
    if target >= 10 and mode == 'train':
        times = 3 if counter_train < 48 else 2
        for i in range(times):
            if clip*(i+1) > len(signal):
                continue
            melspec = librosa.feature.melspectrogram(y=signal[clip*i:clip*(i+1)], n_mels=80,sr=sr)
            audio_features.append(melspec)
            audio_targets.append(target)
            counter_train+=1

    else:
        melspec = librosa.feature.melspectrogram(y=signal[:clip], n_mels=80, sr=sr)
        logspec = melspec
        audio_features.append(logspec)
        audio_targets.append(target)
    print('{}_P feature done'.format(number))

In [22]:
counter_train = 0
audio_features_train = []
audio_ctargets_train = []

audio_features_test = []
audio_ctargets_test = []

In [23]:
for index in range(len(train_split_num)):
    extract_features(train_split_num[index], audio_features_train, train_split_clabel[index], audio_ctargets_train, 'train')

303_P feature done
304_P feature done
305_P feature done
310_P feature done
312_P feature done
313_P feature done
315_P feature done
316_P feature done
317_P feature done
318_P feature done
319_P feature done
320_P feature done
321_P feature done
322_P feature done
324_P feature done
325_P feature done
326_P feature done
327_P feature done
328_P feature done
330_P feature done
333_P feature done
336_P feature done
338_P feature done
339_P feature done
340_P feature done
341_P feature done
343_P feature done
344_P feature done
345_P feature done
347_P feature done
348_P feature done
350_P feature done
351_P feature done
352_P feature done
353_P feature done
355_P feature done
356_P feature done
357_P feature done
358_P feature done
360_P feature done
362_P feature done
363_P feature done
364_P feature done
366_P feature done
368_P feature done
369_P feature done
370_P feature done
371_P feature done
372_P feature done
374_P feature done
375_P feature done
376_P feature done
379_P featur

In [221]:
print("Count 0: ", audio_ctargets_train.count(0))
print("Count 1: ", audio_ctargets_train.count(1))

Count 0:  77
Count 1:  76


In [24]:
for index in range(len(test_split_num)):
    extract_features(test_split_num[index], audio_features_test, test_split_clabel[index], audio_ctargets_test, 'test')

302_P feature done
307_P feature done
331_P feature done
335_P feature done
346_P feature done
367_P feature done
377_P feature done
381_P feature done
382_P feature done
388_P feature done
389_P feature done
390_P feature done
395_P feature done
403_P feature done
404_P feature done
406_P feature done
413_P feature done
417_P feature done
418_P feature done
420_P feature done
422_P feature done
436_P feature done
439_P feature done
440_P feature done
451_P feature done
458_P feature done
472_P feature done
476_P feature done
477_P feature done
482_P feature done
483_P feature done
484_P feature done
489_P feature done
490_P feature done
492_P feature done


In [31]:
np.savez('lab2/train_samples_reg_lab2_1.npz', audio_features_train)
np.savez('lab2/test_samples_reg_lab2_1.npz', audio_features_test)
np.savez('lab2/train_labels_reg_lab2_1.npz', audio_ctargets_train)
np.savez('lab2/test_labels_reg_lab2_1.npz', audio_ctargets_test)

In [352]:
np.savez('lab2/train_samples_cla_lab2_1.npz', audio_features_train)
np.savez('lab2/test_samples_cla_lab2_1.npz', audio_features_test)
np.savez('lab2/train_labels_cla_lab2_1.npz', audio_ctargets_train)
np.savez('lab2/test_labels_cla_lab2_1.npz', audio_ctargets_test)

## classification

In [3]:
features_train = np.load('lab2/train_samples_cla_lab2_1.npz', allow_pickle=True)['arr_0']
features_test = np.load('lab2/test_samples_cla_lab2_1.npz', allow_pickle=True)['arr_0']
targets_train = np.load('lab2/train_labels_cla_lab2_1.npz', allow_pickle=True)['arr_0']
ctargets_test = np.load('lab2/test_labels_cla_lab2_1.npz', allow_pickle=True)['arr_0']

In [4]:
X_train = np.array(features_train)
Y_train = np.array(targets_train)
X_test = np.array(features_test)
Y_test = np.array(ctargets_test)
X_train = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_train])
X_test = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_test])
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(153, 80, 469)
(153,)
(35, 80, 469)
(35,)


In [5]:
train_y = to_categorical(Y_train)
test_y = to_categorical(Y_test)

In [13]:
input_shape = (80, 469, 1)

model = Sequential()
model.add(Conv2D(32, (1, 7), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(4, 3), strides=(1, 3)))
model.add(Conv2D(32, (1, 7), activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer=Adadelta(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
history = model.fit(X_train, train_y, batch_size=4, epochs=30)

Epoch 1/30
39/39 [==============================] - 2s 12ms/step - loss: 0.6929 - accuracy: 0.5098
Epoch 2/30
39/39 [==============================] - 0s 11ms/step - loss: 0.6908 - accuracy: 0.6667
Epoch 3/30
39/39 [==============================] - 0s 10ms/step - loss: 0.6895 - accuracy: 0.6601
Epoch 4/30
39/39 [==============================] - 0s 11ms/step - loss: 0.6863 - accuracy: 0.7712
Epoch 5/30
39/39 [==============================] - 0s 10ms/step - loss: 0.6842 - accuracy: 0.7451
Epoch 6/30
39/39 [==============================] - 0s 11ms/step - loss: 0.6808 - accuracy: 0.7778
Epoch 7/30
39/39 [==============================] - 0s 10ms/step - loss: 0.6771 - accuracy: 0.7320
Epoch 8/30
39/39 [==============================] - 0s 11ms/step - loss: 0.6722 - accuracy: 0.8039
Epoch 9/30
39/39 [==============================] - 0s 10ms/step - loss: 0.6689 - accuracy: 0.8039
Epoch 10/30
39/39 [==============================] - 0s 10ms/step - loss: 0.6653 - accuracy: 0.7451
Epoch 11/

In [183]:
y_pred = model.predict(X_test)
predicted_1 = [1 if x[1] > x[0] else 0 for x in y_pred]
print(classification_report(Y_test, predicted_1))

2/2 [==============================] - 0s 10ms/step
              precision    recall  f1-score   support

         0.0       0.90      0.78      0.84        23
         1.0       0.67      0.83      0.74        12

    accuracy                           0.80        35
   macro avg       0.78      0.81      0.79        35
weighted avg       0.82      0.80      0.80        35





---



---



## regression

In [32]:
features_train = np.load('lab2/train_samples_reg_lab2_1.npz', allow_pickle=True)['arr_0']
features_test = np.load('lab2/test_samples_reg_lab2_1.npz', allow_pickle=True)['arr_0']
targets_train = np.load('lab2/train_labels_reg_lab2_1.npz', allow_pickle=True)['arr_0']
ctargets_test = np.load('lab2/test_labels_reg_lab2_1.npz', allow_pickle=True)['arr_0']

In [33]:
X_train = np.array(features_train)
Y_train = np.array(targets_train)
X_test = np.array(features_test)
Y_test = np.array(ctargets_test)
X_train = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_train])
X_test = np.array([(X - X.min()) / (X.max() - X.min()) for X in X_test])
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(154, 80, 469)
(154,)
(35, 80, 469)
(35,)


In [35]:
input_shape = (80, 469, 1)

model = Sequential()
model.add(Conv2D(32, (1, 7), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(4, 3), strides=(1, 3)))
model.add(Conv2D(32, (1, 7), activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 3), strides=(1, 3)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adadelta(learning_rate=0.1), loss='mean_squared_error', metrics=['mean_absolute_error'])

history = model.fit(X_train, Y_train, epochs=30, validation_data=(X_test, Y_test), batch_size=4)

loss, mean_absolute_error = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}, Test Mean Absolute Error: {mean_absolute_error}")

Epoch 1/30
39/39 [==============================] - 3s 17ms/step - loss: 55.9505 - mean_absolute_error: 5.9876 - val_loss: 45.0519 - val_mean_absolute_error: 5.5374
Epoch 2/30
39/39 [==============================] - 0s 11ms/step - loss: 38.3315 - mean_absolute_error: 5.2370 - val_loss: 45.0936 - val_mean_absolute_error: 5.9083
Epoch 3/30
39/39 [==============================] - 0s 11ms/step - loss: 40.4764 - mean_absolute_error: 5.1516 - val_loss: 44.2341 - val_mean_absolute_error: 5.8439
Epoch 4/30
39/39 [==============================] - 0s 13ms/step - loss: 36.5719 - mean_absolute_error: 4.9079 - val_loss: 47.7157 - val_mean_absolute_error: 6.1749
Epoch 5/30
39/39 [==============================] - 1s 14ms/step - loss: 33.9918 - mean_absolute_error: 4.7934 - val_loss: 40.6399 - val_mean_absolute_error: 5.2940
Epoch 6/30
39/39 [==============================] - 1s 15ms/step - loss: 33.6480 - mean_absolute_error: 4.8212 - val_loss: 48.4768 - val_mean_absolute_error: 6.2121
Epoch 7/30

In [36]:
y_pred = model.predict(X_test)
print("RMSE = ", np.sqrt(mean_squared_error(Y_test,y_pred)))

2/2 [==============================] - 0s 12ms/step
RMSE =  5.564313440193498
